In [29]:
%pip install azure-ai-ml
%pip install mlflow>=2.5.0
%pip install langchain
%pip install --extra-index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004

In [1]:
import os

import mlflow
import openai
from langchain.chains import RetrievalQA
from langchain.chains.loading import load_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.vectorstores.azuresearch import AzureSearch

_SETTING ENVIRONMENT VARIABLES_


In [2]:
# setting openai creds as env vars
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://<azure-openai-endpoint-name>.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "<azure-openai-api-key>"

# setting acs creds as env vars
os.environ["VECTOR_STORE_ADDRESS"] = "https://<azure-search-endpoint>.search.windows.net"
os.environ["VECTOR_STORE_PASSWORD"] = "<azure-search-key>"
os.environ["INDEX_NAME"] = "<azure-search-index-name>"

In [12]:
# function to load the embeddings
def load_embeddings():
    embeddings = OpenAIEmbeddings(
        deployment="text-embedding-ada-002",
        model="text-embedding-ada-002",
        chunk_size=1,
    )
    return embeddings


# function to load the vectorstore
def load_vectorstore(embeddings):
    # using azure cognitive search as the vector store
    vectorstore = AzureSearch(
        azure_search_endpoint=os.getenv("VECTOR_STORE_ADDRESS"),
        azure_search_key=os.getenv("VECTOR_STORE_PASSWORD"),
        index_name=os.getenv("INDEX_NAME"),
        embedding_function=embeddings.embed_query,
    )
    return vectorstore

_ADDING TEXTS AND METADATA TO THE VECTORSTORE_


In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("./stateoftheunion.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store = load_vectorstore(load_embeddings())
_ = vector_store.add_documents(documents=docs)

_SAVING THE MODEL_


In [16]:

llm = AzureOpenAI(
    deployment_name="text-davinci-003",
)
local_dir = "./mlflow_model"

# creating the retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_store.as_retriever()
)
query = "What did the president say about Ketanji Brown Jackson"
answer = qa.run(query)
print(answer)

# function to load the retreiver
def load_retriever(persist_dir):
    embeddings = load_embeddings()

    vectorstore = load_vectorstore(embeddings)

    # returning the retriever object
    return vectorstore.as_retriever()

# saving the model
saved_model = mlflow.langchain.save_model(
    qa,
    local_dir,
    loader_fn=load_retriever,
    extra_pip_requirements=[
        "--extra-index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004",
        "mlflow>=2.5.0",
    ],
)

 The president said that Ketanji Brown Jackson is one of our nation's top legal minds and that she will continue Justice Breyer’s legacy of excellence.


c:\Users\RM\.conda\envs\aml-lc_env\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\RM\.conda\envs\aml-lc_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


_LOADING THE MODEL_


In [17]:
model: RetrievalQA = mlflow.pyfunc.load_model(local_dir)
query = "What did the president say about Ketanji Brown Jackson"
input_row = {"query": query}
answer = model.predict([input_row])
print(answer)

[" The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."]


**_Below is the code to deploy the model in azureml_**


In [18]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()

_CREATING AN INSTANCE OF MLCLIENT_


In [28]:
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

Found the config file in: .\config.json


_REGISTERING THE MODEL_


In [21]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

model_name = "aml-langchain-model"
model_local_path = "./mlflow_model"
model = ml_client.models.create_or_update(
    Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
)


Uploading mlflow_model (0.0 MBs): 100%|##########| 4317/4317 [00:02<00:00, 1697.71it/s]




_SETTING UP ENVIRONMENT VARIABLES FOR OPENAI AND AZURE COGNITIVE SEARCH_


In [22]:
openai_env_vars = {
    "OPENAI_API_TYPE": os.getenv("OPENAI_API_TYPE"),
    "OPENAI_API_VERSION": os.getenv("OPENAI_API_VERSION"),
    "OPENAI_API_BASE": os.getenv("OPENAI_API_BASE") ,
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")
}

acs_env_vars = {
    "VECTOR_STORE_ADDRESS": os.getenv("VECTOR_STORE_ADDRESS"),
    "VECTOR_STORE_PASSWORD": os.getenv("VECTOR_STORE_PASSWORD"),
    "INDEX_NAME": os.getenv("INDEX_NAME"),
}

_CREATING THE ENDPOINT WHERE THE MODEL WILL BE DEPLOYED_


In [23]:
endpoint_name = "aml-langchain-"

import datetime

endpoint_name = endpoint_name + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")
ml_client.online_endpoints.begin_create_or_update(endpoint)

_SETTING UP DEPLOYMENT PARAMETERS_


In [24]:
DEPLOYMENT_NAME = "aml-langchain-deployment"
deployment = ManagedOnlineDeployment(
    name=DEPLOYMENT_NAME,
    endpoint_name=endpoint_name,
    model=model,
    environment_variables={
        **openai_env_vars,
        **acs_env_vars,
    },
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

_DEPLOYING TO THE ENDPOINT_


In [25]:
ml_client.online_deployments.begin_create_or_update(deployment)

Check: endpoint aml-langchain-07191453558955 exists


.......................................................................................................

_UPDATING TRAFFIC TO THE ENDPOINT_


In [26]:
endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

_TESTING THE RESULTS_


In [27]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="request.json",
)

'[" The president said that she is a Circuit Court of Appeals Judge and one of our nation\\u2019s top legal minds who will continue Justice Breyer\\u2019s legacy of excellence."]'